In [ ]:
!pip install -q pyngrok google-colab pdfplumber google-generativeai \
langchain langchain-google-genai langchain_community \
sentence-transformers chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.3 MB/s eta 0:00:

In [ ]:
# Install all dependencies in one go
!pip install streamlit youtube-transcript-api gtts pydub python-multipart validators wikipedia anthropic openai requests beautifulsoup4 PyPDF2 python-docx pyngrok -q

# Install additional audio processing libraries
!apt-get install -y espeak espeak-data libespeak1 libespeak-dev festival festvox-kallpc16k -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.2/287.2 kB 6.4 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  alsa-utils festlex-cmu festlex-poslex libatopology2 libestools2.5
  libfftw3-single3 libportaudio2 libsonic0 sgml-base
Suggested packages:
  dialog pidgin-festival festival-freebsoft-utils libfftw3-bin libfftw3-dev
  sgml-base-doc
The following NEW packages will be installed:
  alsa-utils espeak espeak-data festival festlex-cmu festlex-poslex
  festvox-kallpc16k libatopology2 libespeak-dev libespeak1 libestools2.5
  libfftw3-single3 libportaudio2 libsonic0 sgml-base
0 upgraded, 15 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.1 MB of archives.
After this operation, 30.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 sgml-base all 1.30 [12.5 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liba

In [ ]:
# Set up any API keys
import os

import threading
import subprocess
import time
from pyngrok import ngrok
from google.colab import files
import os
import pdfplumber
import google.generativeai as genai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# Uncomment and add your API keys if needed
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')
# os.environ["ANTHROPIC_API_KEY"] = "your-api-key-here"


In [ ]:
def upload_pdf(pdf_path):
    """
    Function to handle PDF uploads.

    Args:
        pdf_path (str): Path to the PDF file

    Returns:
        str: PDF file path if successful
    """
    try:
        # In a real application with Streamlit, you would use:
        # uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")
        # But for this notebook, we'll just verify the file exists

        if os.path.exists(pdf_path):
            print(f"PDF file found at: {pdf_path}")
            return pdf_path
        else:
            print(f"Error: File not found at {pdf_path}")
            return None
    except Exception as e:
        print(f"Error uploading PDF: {e}")
        return None
def parse_pdf(pdf_path):
    """
    Function to extract text from PDF files.

    Args:
        pdf_path (str): Path to the PDF file

    Returns:
        str: Extracted text from the PDF
    """
    try:
        text = ""

        # Using pdfplumber to extract text
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() + "\n"

        # Save the extracted text to a file (optional)
        text_file_path = pdf_path.replace('.pdf', '.txt')
        with open(text_file_path, 'w', encoding='utf-8') as f:
            f.write(text)

        print(f"PDF parsed successfully, extracted {len(text)} characters")
        return text
    except Exception as e:
        print(f"Error parsing PDF: {e}")
        return None

def create_document_chunks(text):
    """
    Function to split the document text into smaller chunks for processing.

    Args:
        text (str): The full text from the PDF

    Returns:
        list: List of text chunks
    """
    try:
        # Initialize the text splitter
        # We can tune these parameters based on our needs and model constraints
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=250,        # Size of each chunk in characters
            chunk_overlap=25,      # Overlap between chunks to maintain context
            length_function=len,
            separators=["\n\n", "\n", " ", ""]  # Hierarchy of separators to use when splitting
        )

        # Split the text into chunks
        chunks = text_splitter.split_text(text)

        print(f"Document split into {len(chunks)} chunks")
        return chunks
    except Exception as e:
        print(f"Error creating document chunks: {e}")
        return []
# Add this function definition to your app.py file

def init_embedding_model():
    """Initializes and returns the embedding model."""
    try:
        embedding_model = GoogleGenerativeAIEmbeddings(
            model="models/text-embedding-004"
        )
        return embedding_model
    except Exception as e:
        st.error(f"Error initializing embedding model: {str(e)}")
        return None

def embed_and_view(text_chunks):
    """
    Embed document chunks and display their numeric embeddings.

    Args:
        text_chunks (list): List of text chunks from the document
    """
    try:
        # Initialize the Gemini embeddings
        embedding_model = GoogleGenerativeAIEmbeddings(
            model="models/text-embedding-004"  # Specify the Gemini Embedding model
        )

        print("Embedding model initialized successfully")

        # Generate and display embeddings for all chunks
        for i, chunk in enumerate(text_chunks):
            embedding = embedding_model.embed_query(chunk)
            print(f"Chunk {i} Embedding:\n{embedding}\n")

    except Exception as e:
        print(f"Error embedding documents: {e}")

# Example usage
sample_chunks = ["This is the first chunk.", "This is the second chunk.", "And this is the third chunk."]
embed_and_view(sample_chunks)

def embed_documents(text_chunks):
    """
    Function to generate embeddings for the text chunks.

    Args:
        text_chunks (list): List of text chunks from the document

    Returns:
        object: Embedding model for further use
    """
    try:
        # Initialize the Gemini embeddings
        embedding_model = GoogleGenerativeAIEmbeddings(
            model="models/text-embedding-004"  # Specify the Gemini Embedding model
        )

        print("Embedding model initialized successfully")
        return embedding_model, text_chunks
    except Exception as e:
        print(f"Error embedding documents: {e}")
        return None, None

def store_embeddings(embedding_model, text_chunks):
    """
    Function to store document embeddings in ChromaDB.

    Args:
        embedding_model: The embedding model to use
        text_chunks (list): List of text chunks to embed and store

    Returns:
        object: Vector store for retrieval
    """
    try:
        # Create a vector store from the documents
        vectorstore = Chroma.from_texts(
            texts=text_chunks,
            embedding=embedding_model,
            persist_directory="./chroma_db"  # Directory to persist the database
        )

        # Persist the vector store to disk
        vectorstore.persist()

        print(f"Successfully stored {len(text_chunks)} document chunks in ChromaDB")
        return vectorstore
    except Exception as e:
        print(f"Error storing embeddings: {e}")
        return None


def embed_query(query, embedding_model):
    """
    Function to embed the user's query.

    Args:
        query (str): User's question
        embedding_model: The embedding model to use

    Returns:
        list: Embedded query vector
    """
    try:
        # Generate embedding for the query
        query_embedding = embedding_model.embed_query(query)

        print("Query embedded successfully")
        return query_embedding
    except Exception as e:
        print(f"Error embedding query: {e}")
        return None

def retrieve_relevant_chunks(vectorstore, query, embedding_model, k=3):
    """
    Function to retrieve the most relevant document chunks for a query.

    Args:
        vectorstore: The ChromaDB vector store
        query (str): User's question
        embedding_model: The embedding model
        k (int): Number of chunks to retrieve

    Returns:
        list: List of relevant document chunks
    """
    try:
        # Create a retriever from the vector store
        retriever = vectorstore.as_retriever(
            search_type="similarity",  # Can also use "mmr" for Maximum Marginal Relevance
            search_kwargs={"k": k}     # Number of documents to retrieve
        )

        # Retrieve relevant chunks
        relevant_chunks = retriever.get_relevant_documents(query)

        print(f"Retrieved {len(relevant_chunks)} relevant document chunks")
        return relevant_chunks
    except Exception as e:
        print(f"Error retrieving chunks: {e}")
        return []


def get_context_from_chunks(relevant_chunks, splitter="\n\n---\n\n"):
    """
    Extract page_content from document chunks and join them with a splitter.

    Args:
        relevant_chunks (list): List of document chunks from retriever
        splitter (str): String to use as separator between chunk contents

    Returns:
        str: Combined context from all chunks
    """
    # Extract page_content from each chunk
    chunk_contents = []

    for i, chunk in enumerate(relevant_chunks):
        if hasattr(chunk, 'page_content'):
            # Add a chunk identifier to help with tracing which chunk provided what information
            chunk_text = f"[Chunk {i+1}]: {chunk.page_content}"
            chunk_contents.append(chunk_text)

    # Join all contents with the splitter
    combined_context = splitter.join(chunk_contents)

    return combined_context



Embedding model initialized successfully
Chunk 0 Embedding:
[0.027561187744140625, -0.030431516468524933, -0.05948604270815849, -0.043801628053188324, 0.03602953255176544, 0.005739286541938782, 0.009846016764640808, 0.040446169674396515, -0.014708821661770344, 0.004710732027888298, -0.022227749228477478, 0.07038010656833649, -0.007956956513226032, 0.00843250285834074, 0.017124926671385765, -0.0008936881786212325, -0.004870439879596233, -0.006309256888926029, -0.09675060212612152, 0.0011923924321308732, 0.021784083917737007, -0.023273255676031113, -0.022510481998324394, -0.08129142224788666, 0.012183766812086105, 0.029281053692102432, 0.05132291838526726, -0.04530042037367821, 0.06540543586015701, -0.010832075029611588, 0.03330257162451744, 0.022882545366883278, -0.008646462112665176, -0.04968956485390663, 0.003475470934063196, 0.021871499717235565, -0.04579205438494682, 0.039709657430648804, -0.009846191853284836, -0.020981719717383385, -0.05180809274315834, 0.08008384704589844, -0.050

In [ ]:
%%writefile app.py
import streamlit as st
import os
import tempfile
import json
import re
from typing import List, Dict, Any
import pandas as pd
from youtube_transcript_api import YouTubeTranscriptApi
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

# Import your existing functions


# Page configuration
st.set_page_config(
    page_title="AI Study Helper",
    page_icon="📚",
    layout="wide"
)

# Session state initialization
if "conversation" not in st.session_state:
    st.session_state.conversation = []
if "vectorstore" not in st.session_state:
    st.session_state.vectorstore = None
if "embedding_model" not in st.session_state:
    st.session_state.embedding_model = None
if "processed_files" not in st.session_state:
    st.session_state.processed_files = []
if "flashcards" not in st.session_state:
    st.session_state.flashcards = []
if "current_card" not in st.session_state:
    st.session_state.current_card = 0
if "show_answer" not in st.session_state:
    st.session_state.show_answer = False
if "youtube_content" not in st.session_state:
    st.session_state.youtube_content = ""

# YouTube API Functions
def extract_video_id(youtube_url: str) -> str:
    """Extract video ID from YouTube URL"""
    patterns = [
        r'(?:youtube\.com\/watch\?v=|youtu\.be\/|youtube\.com\/embed\/)([^&\n?#]+)',
        r'youtube\.com\/watch\?.*v=([^&\n?#]+)'
    ]

    for pattern in patterns:
        match = re.search(pattern, youtube_url)
        if match:
            return match.group(1)
    return None

def get_youtube_transcript(video_id: str) -> str:
    """Get transcript from YouTube video"""
    try:
        transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

        try:
            transcript = transcript_list.find_transcript(['en'])
            transcript_data = transcript.fetch()
        except:
            transcript = transcript_list.find_generated_transcripts(['en'])
            transcript_data = transcript.fetch()

        full_transcript = " ".join([item['text'] for item in transcript_data])
        return full_transcript

    except Exception as e:
        st.error(f"Error getting transcript: {str(e)}")
        return None

def search_related_videos(query: str) -> List[Dict]:
    """Generate related video search suggestions using AI"""
    try:
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-thinking-exp-01-21",
            temperature=0.3
        )

        prompt = f"""
        Based on the topic: "{query}"

        Generate 5 educational YouTube search queries that would help someone learn about this topic.
        Format as a simple list, one per line, no numbers or bullets.
        Focus on educational, tutorial, or explanatory content.
        Keep each suggestion under 50 characters.
        """

        response = llm.invoke(prompt)
        suggestions = response.content.strip().split('\n')

        return [{"title": suggestion.strip(), "search_query": suggestion.strip()}
                for suggestion in suggestions if suggestion.strip()]
    except Exception as e:
        st.error(f"Error generating suggestions: {str(e)}")
        return [{"title": f"Learn about {query}", "search_query": query}]

# Flashcard Generation Functions
def generate_flashcards_from_content(content: str, num_cards: int = 10) -> List[Dict]:
    """Generate flashcards from content using Gemini AI"""
    try:
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-thinking-exp-01-21",
            temperature=0.3
        )

        prompt = f"""
        Create {num_cards} educational flashcards from the following content.
        Return ONLY a valid JSON array with the following structure:
        [
            {{"question": "Question text", "answer": "Answer text", "difficulty": "Easy"}},
            {{"question": "Question text", "answer": "Answer text", "difficulty": "Medium"}},
            {{"question": "Question text", "answer": "Answer text", "difficulty": "Hard"}}
        ]

        Guidelines:
        - Create clear, concise questions
        - Provide comprehensive but not overly long answers
        - Mix different types of questions (definitions, concepts, applications)
        - Assign appropriate difficulty levels (Easy, Medium, Hard)
        - Focus on key learning points
        - Return ONLY the JSON array, no other text

        Content:
        {content[:4000]}
        """

        response = llm.invoke(prompt)
        response_text = response.content.strip()

        # Clean the response to extract JSON
        if response_text.startswith('```json'):
            response_text = response_text[7:-3]
        elif response_text.startswith('```'):
            response_text = response_text[3:-3]

        # Find JSON array in the response
        start_idx = response_text.find('[')
        end_idx = response_text.rfind(']') + 1

        if start_idx != -1 and end_idx != -1:
            json_text = response_text[start_idx:end_idx]
            flashcards = json.loads(json_text)
            return flashcards
        else:
            raise ValueError("No JSON array found in response")

    except Exception as e:
        st.error(f"Error generating flashcards: {str(e)}")
        # Fallback: create simple flashcards
        return [
            {
                "question": "What is the main topic of this content?",
                "answer": "Please refer to the uploaded content for details.",
                "difficulty": "Medium"
            }
        ]

def display_flashcard_interface():
    """Display interactive flashcard interface"""
    if not st.session_state.flashcards:
        st.warning("No flashcards available. Generate flashcards first!")
        return

    flashcards = st.session_state.flashcards
    current_card = st.session_state.current_card
    total_cards = len(flashcards)

    st.subheader("📚 Interactive Flashcards")

    # Card navigation
    col1, col2, col3 = st.columns([1, 2, 1])

    with col1:
        if st.button("← Previous", disabled=(current_card == 0)):
            st.session_state.current_card = max(0, current_card - 1)
            st.session_state.show_answer = False
            st.rerun()

    with col2:
        st.write(f"Card {current_card + 1} of {total_cards}")
        difficulty = flashcards[current_card].get('difficulty', 'Medium')
        if difficulty == 'Easy':
            st.success(f"Difficulty: {difficulty}")
        elif difficulty == 'Medium':
            st.warning(f"Difficulty: {difficulty}")
        else:
            st.error(f"Difficulty: {difficulty}")

    with col3:
        if st.button("Next →", disabled=(current_card == total_cards - 1)):
            st.session_state.current_card = min(total_cards - 1, current_card + 1)
            st.session_state.show_answer = False
            st.rerun()

    # Display flashcard
    st.markdown("---")

    # Question
    st.markdown("### 🤔 Question:")
    st.markdown(f"**{flashcards[current_card]['question']}**")

    # Show/Hide answer button
    if st.button("Show Answer" if not st.session_state.show_answer else "Hide Answer"):
        st.session_state.show_answer = not st.session_state.show_answer
        st.rerun()

    # Answer
    if st.session_state.show_answer:
        st.markdown("### ✅ Answer:")
        st.info(flashcards[current_card]['answer'])

    # Progress bar
    progress = (current_card + 1) / total_cards
    st.progress(progress)

def main():
    # Sidebar
    with st.sidebar:
        st.title("🤖 AI Study Helper")
        st.subheader("Configuration")

        # API Key input
        api_key = st.text_input("Enter Gemini API Key:", type="password")

        if api_key:
            os.environ["GOOGLE_API_KEY"] = api_key
            if st.button("Set API Key"):
                st.success("API Key set successfully!")

        st.divider()

        # Navigation
        page = st.selectbox("Choose Feature:", [
            "📄 PDF Analysis",
            "🎥 YouTube Integration",
            "🃏 Flashcard Generator",
            "💬 Chat with Documents"
        ])

        st.divider()

        # File uploader (always visible)
        st.subheader("📄 Upload Documents")
        uploaded_files = st.file_uploader("Upload PDF files", type="pdf", accept_multiple_files=True)

        if uploaded_files and st.button("Process Documents"):
            process_documents(uploaded_files)

        # Display processed files
        if st.session_state.processed_files:
            st.subheader("✅ Processed Documents")
            for file in st.session_state.processed_files:
                st.write(f"- {file}")

    # Main content area based on selected page
    if page == "📄 PDF Analysis":
        display_pdf_analysis()
    elif page == "🎥 YouTube Integration":
        display_youtube_integration()
    elif page == "🃏 Flashcard Generator":
        display_flashcard_generator()
    elif page == "💬 Chat with Documents":
        display_chat_interface()

def display_pdf_analysis():
    st.title("📄 PDF Document Analysis")

    if not st.session_state.processed_files:
        st.info("Upload and process PDF documents to start analysis.")
        return

    st.success(f"Processed {len(st.session_state.processed_files)} documents successfully!")

    # Quick analysis
    if st.button("Generate Quick Summary"):
        if st.session_state.vectorstore:
            with st.spinner("Generating summary..."):
                summary_query = "What are the main topics and key concepts in this document?"
                relevant_chunks = retrieve_relevant_chunks(
                    st.session_state.vectorstore,
                    summary_query,
                    st.session_state.embedding_model
                )
                context = get_context_from_chunks(relevant_chunks)

                prompt = f"""
                Based on the following content, provide a comprehensive summary of the main topics and key concepts:

                {context}

                Summary:
                """

                summary = generate_response(prompt)
                st.markdown("### 📋 Document Summary")
                st.write(summary)

def display_youtube_integration():
    st.title("🎥 YouTube Video Integration")

    # YouTube URL input
    youtube_url = st.text_input("Enter YouTube Video URL:", placeholder="https://www.youtube.com/watch?v=...")

    if youtube_url:
        video_id = extract_video_id(youtube_url)

        if video_id:
            st.success(f"Video ID extracted: {video_id}")

            if st.button("Get Video Transcript"):
                with st.spinner("Fetching transcript..."):
                    transcript = get_youtube_transcript(video_id)

                    if transcript:
                        st.session_state.youtube_content = transcript
                        st.success("Transcript fetched successfully!")

                        with st.expander("View Transcript"):
                            st.text_area("Transcript:", transcript, height=200)

                        # Option to add to vector database
                        if st.button("Add to Knowledge Base"):
                            with st.spinner("Processing transcript..."):
                                # Create chunks from transcript
                                chunks = create_document_chunks(transcript)

                                if chunks and hasattr(st.session_state, 'embedding_model') and st.session_state.embedding_model:
                                    # Add to existing vector store or create new one
                                    if st.session_state.vectorstore:
                                        # Add to existing
                                        st.session_state.vectorstore.add_texts(chunks)
                                        st.success("Transcript added to knowledge base!")
                                    else:
                                        # Create new vector store
                                        st.session_state.vectorstore = store_embeddings(
                                            st.session_state.embedding_model,
                                            chunks
                                        )
                                        st.success("New knowledge base created with transcript!")
                                else:
                                    st.error("Please process a PDF document first to initialize the embedding model.")
        else:
            st.error("Invalid YouTube URL. Please check the URL format.")

    # Related video suggestions
    if st.session_state.processed_files:
        st.subheader("🔍 Find Related Videos")

        if st.button("Generate Video Search Suggestions"):
            # Use the first processed file as context
            sample_query = f"educational videos about {st.session_state.processed_files[0]}"

            with st.spinner("Generating suggestions..."):
                suggestions = search_related_videos(sample_query)

                st.markdown("### 🎯 Suggested Video Searches:")
                for i, suggestion in enumerate(suggestions, 1):
                    st.write(f"{i}. {suggestion['title']}")

def display_flashcard_generator():
    st.title("🃏 Flashcard Generator")

    # Choose content source
    content_source = st.radio("Choose content source:", [
        "📄 Processed Documents",
        "🎥 YouTube Transcript",
        "✍️ Custom Text"
    ])

    content_to_use = ""

    if content_source == "📄 Processed Documents":
        if not st.session_state.vectorstore:
            st.warning("No processed documents available. Please upload and process documents first.")
            return

        # Get content from documents
        if st.button("Use Document Content"):
            # Sample query to get representative content
            sample_query = "main concepts and important topics"
            relevant_chunks = retrieve_relevant_chunks(
                st.session_state.vectorstore,
                sample_query,
                st.session_state.embedding_model,
                k=5
            )
            content_to_use = get_context_from_chunks(relevant_chunks)
            st.text_area("Content Preview:", content_to_use[:500] + "...", height=100)

    elif content_source == "🎥 YouTube Transcript":
        if not st.session_state.youtube_content:
            st.warning("No YouTube transcript available. Please fetch a transcript first.")
            return
        content_to_use = st.session_state.youtube_content
        st.text_area("Content Preview:", content_to_use[:500] + "...", height=100)

    elif content_source == "✍️ Custom Text":
        content_to_use = st.text_area("Enter your content:", height=200)

    # Flashcard generation settings
    col1, col2 = st.columns(2)
    with col1:
        num_cards = st.slider("Number of flashcards:", 5, 20, 10)
    with col2:
        difficulty_filter = st.selectbox("Difficulty focus:", ["Mixed", "Easy", "Medium", "Hard"])

    # Generate flashcards
    if st.button("🎯 Generate Flashcards") and content_to_use:
        with st.spinner("Generating flashcards..."):
            flashcards = generate_flashcards_from_content(content_to_use, num_cards)

            # Filter by difficulty if needed
            if difficulty_filter != "Mixed":
                flashcards = [card for card in flashcards if card.get('difficulty') == difficulty_filter]

            st.session_state.flashcards = flashcards
            st.session_state.current_card = 0
            st.session_state.show_answer = False

            st.success(f"Generated {len(flashcards)} flashcards!")

    # Display flashcards if they exist
    if st.session_state.flashcards:
        st.markdown("---")
        display_flashcard_interface()

def display_chat_interface():
    st.title("💬 Chat with Your Documents")

    if not st.session_state.vectorstore:
        st.info("Please upload and process documents to start chatting.")
        return

    # Chat interface
    for message in st.session_state.conversation:
        if message["role"] == "user":
            with st.chat_message("user"):
                st.write(message["content"])
        else:
            with st.chat_message("assistant"):
                st.write(message["content"])

    # User input
    user_query = st.chat_input("Ask a question about your documents...")
    if user_query:
        handle_user_query(user_query)

def process_documents(uploaded_files):
    """Process uploaded PDF documents and create the vector store"""
    try:
        progress_bar = st.sidebar.progress(0)
        status_text = st.sidebar.empty()

        # Initialize embedding model using the dedicated function
        if st.session_state.embedding_model is None:
            status_text.text("Initializing embedding model...")
            st.session_state.embedding_model = init_embedding_model()
            if st.session_state.embedding_model is None:
                st.sidebar.error("Failed to initialize embedding model. Check your API key.")
                return

        # Process each uploaded file
        all_chunks = []
        processed_file_names = []

        for i, uploaded_file in enumerate(uploaded_files):
            progress = (i / len(uploaded_files)) * 100
            progress_bar.progress(int(progress))
            status_text.text(f"Processing {uploaded_file.name}...")

            # Save uploaded file temporarily
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(uploaded_file.getvalue())
                pdf_path = tmp_file.name

            # Parse PDF to extract text
            text = parse_pdf(pdf_path)
            if not text:
                st.sidebar.warning(f"Failed to extract text from {uploaded_file.name}")
                continue

            # Create document chunks
            chunks = create_document_chunks(text)
            if not chunks:
                st.sidebar.warning(f"Failed to create chunks from {uploaded_file.name}")
                continue

            all_chunks.extend(chunks)
            processed_file_names.append(uploaded_file.name)

            # Clean up temporary file
            os.unlink(pdf_path)

        # Store embeddings in vector database
        if all_chunks:
            status_text.text("Creating vector database...")
            # Pass the initialized embedding model to store_embeddings
            vectorstore = store_embeddings(st.session_state.embedding_model, all_chunks)

            if vectorstore:
                st.session_state.vectorstore = vectorstore
                st.session_state.processed_files = processed_file_names
                status_text.text("Processing complete!")
                st.sidebar.success(f"Successfully processed {len(processed_file_names)} documents")
            else:
                st.sidebar.error("Failed to create vector database")
        else:
            st.sidebar.error("No valid chunks extracted from documents")

        progress_bar.empty()
        status_text.empty()

    except Exception as e:
        st.sidebar.error(f"Error processing documents: {str(e)}")

def handle_user_query(query):
    """Process a user query and display the response"""
    if st.session_state.vectorstore is None:
        st.error("Please process documents before asking questions")
        return

    # Add user message to conversation
    st.session_state.conversation.append({"role": "user", "content": query})

    with st.spinner("🤔 Thinking..."):
        try:
            # Retrieve relevant chunks
            relevant_chunks = retrieve_relevant_chunks(
                st.session_state.vectorstore,
                query,
                st.session_state.embedding_model,
                k=3
            )

            # Get context from chunks
            context = get_context_from_chunks(relevant_chunks)

            # Generate response
            final_prompt = f"""
            You are an AI assistant helping with document analysis and learning.

            Context from documents:
            {context}

            User question: {query}

            Please provide a helpful and accurate response based on the context provided.
            """

            response = generate_response(final_prompt)

            # Add assistant response to conversation
            st.session_state.conversation.append({"role": "assistant", "content": response})

            st.rerun()

        except Exception as e:
            error_msg = f"Error generating response: {str(e)}"
            st.session_state.conversation.append({"role": "assistant", "content": error_msg})
            st.rerun()

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
# Create .streamlit directory and config
!mkdir -p .streamlit

config_content = '''
[server]
headless = true
enableCORS = false
enableXsrfProtection = false
port = 8501

[browser]
gatherUsageStats = false
'''

with open('.streamlit/config.toml', 'w') as f:
    f.write(config_content)

In [ ]:
# Function to run Streamlit
def run_streamlit():
    !nohup streamlit run app.py --server.port 8501 > streamlit.log 2>&1 &

# Start Streamlit in background
import threading
import time
ngrok.kill()
ngrok.set_auth_token(userdata.get('NGROK_API_KEY'))
# Kill any existing streamlit processes
!pkill -f streamlit

# Start streamlit in background thread
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.daemon = True
streamlit_thread.start()

# Wait a moment for streamlit to start
time.sleep(5)

# Create ngrok tunnel
public_url = ngrok.connect(8501)
print(f"🚀 Your AI Study Helper is running at: {public_url}")
print("Click the link above to access your app!")

🚀 Your AI Study Helper is running at: NgrokTunnel: "https://7ff2-34-74-181-254.ngrok-free.app" -> "http://localhost:8501"
Click the link above to access your app!


In [ ]:
!ps -ef | grep streamlit


root       23434       1  1 17:48 ?        00:00:05 /usr/bin/python3 /usr/local/bin/streamlit run app.py --server.port 8501
root       24659    2814  0 17:53 ?        00:00:00 /bin/bash -c ps -ef | grep streamlit
root       24661   24659  0 17:53 ?        00:00:00 grep streamlit


In [ ]:
ngrok.kill()

In [ ]:
!sudo kill -9 23434